<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fairseq-Load-Dataset" data-toc-modified-id="Fairseq-Load-Dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fairseq Load Dataset</a></span></li><li><span><a href="#Fairseq-Scoring" data-toc-modified-id="Fairseq-Scoring-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fairseq Scoring</a></span></li><li><span><a href="#Fairseq-Generate-Flow" data-toc-modified-id="Fairseq-Generate-Flow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fairseq Generate Flow</a></span></li></ul></div>

# Fairseq Load Dataset

In [40]:
from torchnlp.datasets import multi30k_dataset

directory = 'data/multi30k'
datasets = multi30k_dataset(directory, train=True, dev=True, test=True)
datasets[2][0]

training.tar.gz: 1.21MB [00:02, 431kB/s]                             
validation.tar.gz: 49.2kB [00:01, 40.5kB/s]                            
mmt16_task1_test.tar.gz: 49.2kB [00:01, 41.0kB/s]                            


{'en': 'A man in an orange hat starring at something.',
 'de': 'Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.'}

In [41]:
!ls data/multi30k

mmt16_task1_test.tar.gz train.de                val.de
test.de                 train.en                val.en
test.en                 training.tar.gz         validation.tar.gz


In [42]:
import subprocess

def run_command(cmd):
    """
    Runs a shell command that returns the output message if any.
    Should the command fail, i.e. exit code isn't 0, the program will exit.

    Parameters
    ----------
    cmd : str

    Returns
    -------
    output : str
    """
    process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE, encoding='utf-8', shell=True)
    (output, error) = process.communicate()
    status = process.returncode
    if status != 0:
        print("ERROR: " + error)
        print("ERROR: failed to run command - " + cmd)
        exit(status)

    return output.strip()

In [43]:
source_lang = 'de'
target_lang = 'en'
fairseq_preprocess_format = {
    'source_lang': source_lang,
    'target_lang': target_lang,
    'destdir': 'data/multi30k/data'# 'data/multi30k.de-en'
}

cmd = """
fairseq-preprocess \
    --source-lang {source_lang} \
    --target-lang {target_lang} \
    --trainpref data/multi30k/train \
    --validpref data/multi30k/val \
    --testpref data/multi30k/test \
    --destdir {destdir} \
    --dataset-impl raw
""".format(**fairseq_preprocess_format)
print(cmd)

run_command(cmd)


fairseq-preprocess     --source-lang de     --target-lang en     --trainpref data/multi30k/train     --validpref data/multi30k/val     --testpref data/multi30k/test     --destdir data/multi30k/data     --dataset-impl raw



"Namespace(alignfile=None, cpu=False, criterion='cross_entropy', dataset_impl='raw', destdir='data/multi30k/data', fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='de', srcdict=None, target_lang='en', task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref='data/multi30k/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, trainpref='data/multi30k/train', user_dir=None, validpref='data/multi30k/val', workers=1)\n| Wrote preprocessed data to data/multi30k/data"

In [58]:
class Args:
    task = 'translation'
    data = 'data/multi30k/data'
    
    left_pad_source = True
    left_pad_target = False
    
    source_lang = None
    target_lang = None
    
    upsample_primary = 1
    dataset_impl = 'raw'
    max_source_positions = 1024
    max_target_positions = 1024

    gen_subset = 'test'
    


args = Args()
task = tasks.setup_task(args)
task

| [de] dictionary: 24896 types
| [en] dictionary: 15464 types


In [73]:
import os
from fairseq.data import indexed_dataset


def split_exists(split, src, tgt, lang, data_path, dataset_impl):
    filename = os.path.join(data_path, '{}.{}-{}.{}'.format(split, src, tgt, lang))
    return indexed_dataset.dataset_exists(filename, impl=dataset_impl)

In [74]:
data_path = args.data
src = args.source_lang
tgt = args.target_lang
split = args.gen_subset
dataset_impl = args.dataset_impl

if split_exists(split, src, tgt, src, data_path, dataset_impl):
    prefix = os.path.join(data_path, '{}.{}-{}.'.format(split, src, tgt))
elif split_exists(split, tgt, src, src, data_path, dataset_impl):
    prefix = os.path.join(data_path, '{}.{}-{}.'.format(split, tgt, src))
else:
    raise ValueError('Dataset not found: {} ({})'.format(split, data_path))

prefix

'data/multi30k/data/test.de-en.'

In [70]:
src_dataset = indexed_dataset.make_dataset(prefix + src, impl=dataset_impl,
                                           fix_lua_indexing=True, dictionary=src_dict)
tgt_dataset = indexed_dataset.make_dataset(prefix + tgt, impl=dataset_impl,
                                           fix_lua_indexing=True, dictionary=tgt_dict)

tgt_dataset

In [ ]:
# self.datasets[split] = load_langpair_dataset(
#     data_path, split, src, self.src_dict, tgt, self.tgt_dict,
#     combine=combine, dataset_impl=self.args.dataset_impl,
#     upsample_primary=self.args.upsample_primary,
#     left_pad_source=self.args.left_pad_source,
#     left_pad_target=self.args.left_pad_target,
#     max_source_positions=self.args.max_source_positions,
#     max_target_positions=self.args.max_target_positions,
# )

# Fairseq Scoring

# Fairseq Generate Flow

In [1]:
import argparse
from fairseq import utils, tasks, checkpoint_utils
from fairseq.meters import StopwatchMeter

In [100]:
class Args:
    task = 'translation'
    left_pad_source = True
    left_pad_target = False
    
    source_lang = None
    target_lang = None
    cpu = False
    replace_unk = None
    
    no_beamable_mm = False # ??
    skip_invalid_size_inputs_valid_test = False # ??
    num_shards = 1
    num_workers = 1
    shard_id = 0

    upsample_primary = 1
    max_source_positions = 1024
    max_target_positions = 1024
    max_tokens = 12000
    max_sentences = 10
    required_batch_size_multiple = 8
    prefix_size = 0

    # input parameters
    data = '/Users/mingyuliu/Desktop/20191021_224310/data'
    gen_subset = 'new'
    dataset_impl = 'raw'
    path = '/Users/mingyuliu/Desktop/20191021_224310/models/checkpoint_best.pt'
    beam = 1
    print_alignment = False
    fp16 = False
    batch_size = 2 # ??
    remove_bpe = None
    quiet = False
    nbest = 1


args = Args()
task = tasks.setup_task(args)
task

| [noisy] dictionary: 212296 types
| [original] dictionary: 121200 types


In [101]:
task.load_dataset(args.gen_subset)

# Set dictionaries
try:
    src_dict = getattr(task, 'source_dictionary', None)
except NotImplementedError:
    src_dict = None

tgt_dict = task.target_dictionary

| /Users/mingyuliu/Desktop/20191021_224310/data new noisy-original 200 examples


In [102]:
# Load ensemble
print('| loading model(s) from {}'.format(args.path))
models, _model_args = checkpoint_utils.load_model_ensemble(
    args.path.split(':'),
    task=task
)
models

| loading model(s) from /Users/mingyuliu/Desktop/20191021_224310/models/checkpoint_best.pt


[FConvModel(
   (encoder): FConvEncoder(
     (embed_tokens): Embedding(212296, 256, padding_idx=1)
     (embed_positions): LearnedPositionalEmbedding(1024, 256, padding_idx=1)
     (fc1): Linear(in_features=256, out_features=256, bias=True)
     (projections): ModuleList(
       (0): None
       (1): None
       (2): None
       (3): None
     )
     (convolutions): ModuleList(
       (0): ConvTBC(256, 512, kernel_size=(3,), padding=(1,))
       (1): ConvTBC(256, 512, kernel_size=(3,), padding=(1,))
       (2): ConvTBC(256, 512, kernel_size=(3,), padding=(1,))
       (3): ConvTBC(256, 512, kernel_size=(3,), padding=(1,))
     )
     (fc2): Linear(in_features=256, out_features=256, bias=True)
   )
   (decoder): FConvDecoder(
     (embed_tokens): Embedding(121200, 256, padding_idx=1)
     (embed_positions): LearnedPositionalEmbedding(1024, 256, padding_idx=1)
     (fc1): Linear(in_features=256, out_features=256, bias=True)
     (projections): ModuleList(
       (0): None
       (1): Non

In [103]:
import torch

# Optimize ensemble for generation
use_cuda = torch.cuda.is_available() and not args.cpu

for model in models:
    model.make_generation_fast_(
        beamable_mm_beam_size=None if args.no_beamable_mm else args.beam,
        need_attn=args.print_alignment,
    )
    if args.fp16:
        model.half()
    if use_cuda:
        model.cuda()

In [104]:
# Load alignment dictionary for unknown word replacement
# (None if no unknown word replacement, empty if no path to align dictionary)
align_dict = utils.load_align_dict(args.replace_unk)

In [105]:
# Load dataset (possibly sharded)
itr = task.get_batch_iterator(
    dataset=task.dataset(args.gen_subset),
    max_tokens=args.max_tokens,
    max_sentences=args.max_sentences,
    max_positions=utils.resolve_max_positions(
        task.max_positions(),
        *[model.max_positions() for model in models]
    ),
    ignore_invalid_inputs=args.skip_invalid_size_inputs_valid_test,
    required_batch_size_multiple=args.required_batch_size_multiple,
    num_shards=args.num_shards,
    shard_id=args.shard_id,
    num_workers=args.num_workers,
).next_epoch_itr(shuffle=False)
itr

In [106]:
sample = next(iter(itr))
sample

{'id': tensor([108, 116, 117, 120, 154, 112,  74, 147]),
 'nsentences': 8,
 'ntokens': 16,
 'net_input': {'src_tokens': tensor([[     3,      2],
          [     3,      2],
          [  5845,      2],
          [107341,      2],
          [     3,      2],
          [     3,      2],
          [     3,      2],
          [ 53762,      2]]),
  'src_lengths': tensor([2, 2, 2, 2, 2, 2, 2, 2]),
  'prev_output_tokens': tensor([[     2,   2519],
          [     2, 113263],
          [     2,   5255],
          [     2,   1986],
          [     2,   7599],
          [     2,      3],
          [     2,  37025],
          [     2,  38250]])},
 'target': tensor([[  2519,      2],
         [113263,      2],
         [  5255,      2],
         [  1986,      2],
         [  7599,      2],
         [     3,      2],
         [ 37025,      2],
         [ 38250,      2]])}

In [107]:
sample['net_input']['src_tokens'].shape

torch.Size([8, 2])

In [108]:
# Initialize generator
gen_timer = StopwatchMeter()
generator = task.build_generator(args)

In [109]:
sample = utils.move_to_cuda(sample) if use_cuda else sample
# if 'net_input' not in sample:
#     continue

prefix_tokens = None
if args.prefix_size > 0:
    prefix_tokens = sample['target'][:, :args.prefix_size]

In [110]:
hypos = task.inference_step(generator, models, sample, prefix_tokens)
num_generated_tokens = sum(len(h[0]['tokens']) for h in hypos)
hypos

[[{'tokens': tensor([3, 2]),
   'score': -0.15852700173854828,
   'attention': None,
   'alignment': None,
   'positional_scores': tensor([-0.3162, -0.0009])}],
 [{'tokens': tensor([3, 2]),
   'score': -0.15852700173854828,
   'attention': None,
   'alignment': None,
   'positional_scores': tensor([-0.3162, -0.0009])}],
 [{'tokens': tensor([5255,    2]),
   'score': -0.26697462797164917,
   'attention': None,
   'alignment': None,
   'positional_scores': tensor([-0.5308, -0.0031])}],
 [{'tokens': tensor([3, 2]),
   'score': -0.29383397102355957,
   'attention': None,
   'alignment': None,
   'positional_scores': tensor([-0.5864, -0.0013])}],
 [{'tokens': tensor([3, 2]),
   'score': -0.15852700173854828,
   'attention': None,
   'alignment': None,
   'positional_scores': tensor([-0.3162, -0.0009])}],
 [{'tokens': tensor([3, 2]),
   'score': -0.15852700173854828,
   'attention': None,
   'alignment': None,
   'positional_scores': tensor([-0.3162, -0.0009])}],
 [{'tokens': tensor([3, 2]),

In [111]:
i = 0
sample_id = sample['id'][i]


has_target = sample['target'] is not None

In [112]:
src_tokens = utils.strip_pad(sample['net_input']['src_tokens'][i, :], tgt_dict.pad())
target_tokens = None
if has_target:
    target_tokens = utils.strip_pad(sample['target'][i, :], tgt_dict.pad()).int().cpu()
    
print(src_tokens)
print(target_tokens)

tensor([3, 2])
tensor([2519,    2], dtype=torch.int32)


In [113]:
if align_dict is not None:
    src_str = task.dataset(args.gen_subset).src.get_original_text(sample_id)
    target_str = task.dataset(args.gen_subset).tgt.get_original_text(sample_id)
else:
    if src_dict is not None:
        src_str = src_dict.string(src_tokens, args.remove_bpe)
    else:
        src_str = ""
    if has_target:
        target_str = tgt_dict.string(target_tokens, args.remove_bpe, escape_unk=True)

In [114]:
if not args.quiet:
    if src_dict is not None:
        print('S-{}\t{}'.format(sample_id, src_str))
    if has_target:
        print('T-{}\t{}'.format(sample_id, target_str))

S-108	<unk>
T-108	slippers


In [115]:
for j, hypo in enumerate(hypos[i][:min(len(hypos), args.nbest)]):
    hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
        hypo_tokens=hypo['tokens'].int().cpu(),
        src_str=src_str,
        alignment=hypo['alignment'].int().cpu() if hypo['alignment'] is not None else None,
        align_dict=align_dict,
        tgt_dict=tgt_dict,
        remove_bpe=args.remove_bpe,
    )

    if not args.quiet:
        print('H-{}\t{}\t{}'.format(sample_id, hypo['score'], hypo_str))
        print('P-{}\t{}'.format(
            sample_id,
            ' '.join(map(
                lambda x: '{:.4f}'.format(x),
                hypo['positional_scores'].tolist(),
            ))
        ))

        if args.print_alignment:
            print('A-{}\t{}'.format(
                sample_id,
                ' '.join(map(lambda x: str(utils.item(x)), alignment))
            ))

H-108	-0.15852700173854828	<unk>
P-108	-0.3162 -0.0009


In [126]:
hypo_str

'<unk>'

In [116]:
for i, sample_id in enumerate(sample['id'].tolist()):
    has_target = sample['target'] is not None

    # Remove padding
    src_tokens = utils.strip_pad(sample['net_input']['src_tokens'][i, :], tgt_dict.pad())
    target_tokens = None
    if has_target:
        target_tokens = utils.strip_pad(sample['target'][i, :], tgt_dict.pad()).int().cpu()

    # Either retrieve the original sentences or regenerate them from tokens.
    if align_dict is not None:
        src_str = task.dataset(args.gen_subset).src.get_original_text(sample_id)
        target_str = task.dataset(args.gen_subset).tgt.get_original_text(sample_id)
    else:
        if src_dict is not None:
            src_str = src_dict.string(src_tokens, args.remove_bpe)
        else:
            src_str = ""
        if has_target:
            target_str = tgt_dict.string(target_tokens, args.remove_bpe, escape_unk=True)

    if not args.quiet:
        if src_dict is not None:
            print('S-{}\t{}'.format(sample_id, src_str))
        if has_target:
            print('T-{}\t{}'.format(sample_id, target_str))

    # Process top predictions
    for j, hypo in enumerate(hypos[i][:min(len(hypos), args.nbest)]):
        hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
            hypo_tokens=hypo['tokens'].int().cpu(),
            src_str=src_str,
            alignment=hypo['alignment'].int().cpu() if hypo['alignment'] is not None else None,
            align_dict=align_dict,
            tgt_dict=tgt_dict,
            remove_bpe=args.remove_bpe,
        )

        if not args.quiet:
            print('H-{}\t{}\t{}'.format(sample_id, hypo['score'], hypo_str))
            print('P-{}\t{}'.format(
                sample_id,
                ' '.join(map(
                    lambda x: '{:.4f}'.format(x),
                    hypo['positional_scores'].tolist(),
                ))
            ))

            if args.print_alignment:
                print('A-{}\t{}'.format(
                    sample_id,
                    ' '.join(map(lambda x: str(utils.item(x)), alignment))
                ))

S-108	<unk>
T-108	slippers
H-108	-0.15852700173854828	<unk>
P-108	-0.3162 -0.0009
S-116	<unk>
T-116	gipiemme
H-116	-0.15852700173854828	<unk>
P-116	-0.3162 -0.0009
S-117	prep
T-117	prep
H-117	-0.26697462797164917	prep
P-117	-0.5308 -0.0031
S-120	themometer
T-120	thermometer
H-120	-0.29383397102355957	<unk>
P-120	-0.5864 -0.0013
S-154	<unk>
T-154	surfboard
H-154	-0.15852700173854828	<unk>
P-154	-0.3162 -0.0009
S-112	<unk>
T-112	<<unk>>
H-112	-0.15852700173854828	<unk>
P-112	-0.3162 -0.0009
S-74	<unk>
T-74	ritchey
H-74	-0.15852700173854828	<unk>
P-74	-0.3162 -0.0009
S-147	bontrager
T-147	bontrager
H-147	-0.5243253707885742	<unk>
P-147	-1.0468 -0.0018


In [120]:
tgt_dict.unk_index

3

In [122]:
tgt_dict[3]

'<unk>'

In [124]:
tgt_dict.index('<unk>')

3